In [10]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["movie_reviews_db"]
collection = db["movie_reviews"]

data = pd.read_csv("/Users/fatimashamilova/movie_review.csv")

collection.insert_many(data.to_dict("records"))

print("Data inserted successfully!")

Data inserted successfully!


In [12]:
pip uninstall sklearn

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [15]:
for review in collection.find().limit(5):  
    print(review)

{'_id': ObjectId('67466a7f0a17b006abdb0350'), 'fold_id': 0, 'cv_tag': 'cv000', 'html_id': 29590, 'sent_id': 0, 'text': "films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before .", 'tag': 'pos'}
{'_id': ObjectId('67466a7f0a17b006abdb0351'), 'fold_id': 0, 'cv_tag': 'cv000', 'html_id': 29590, 'sent_id': 1, 'text': "for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen .", 'tag': 'pos'}
{'_id': ObjectId('67466a7f0a17b006abdb0352'), 'fold_id': 0, 'cv_tag': 'cv000', 'html_id': 29590, 'sent_id': 2, 'text': 'to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd .', 'tag': 'pos'}
{'

In [18]:

doc = collection.find_one() 
print(doc.keys())  

dict_keys(['_id', 'fold_id', 'cv_tag', 'html_id', 'sent_id', 'text', 'tag'])


In [19]:

documents = [doc['text'] for doc in collection.find()]  
labels = [doc['tag'] for doc in collection.find()]     


print("Sample Text Data:", documents[:5])  
print("Sample Tags:", labels[:5])         

Sample Text Data: ["films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before .", "for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen .", 'to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd .', 'the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes .', "in other words , don't dismiss this film because of its source ."]
Sample Tags: ['pos', 'pos', 'pos', 'pos', 'pos']


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer()

bow_matrix = bow_vectorizer.fit_transform(documents)

print("BoW Matrix Shape:", bow_matrix.shape)  
print("Sample BoW Vector (First Document):", bow_matrix[0].toarray())
print("Feature Names (First 10 Words):", bow_vectorizer.get_feature_names_out()[:10])  

BoW Matrix Shape: (129440, 39659)
Sample BoW Vector (First Document): [[0 0 0 ... 0 0 0]]
Feature Names (First 10 Words): ['00' '000' '0009f' '007' '00s' '03' '04' '05' '05425' '10']


In [23]:

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk

dataset = [
    "films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before .",
    "for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen .",
    "to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd .",
    "the book ( or ' graphic novel , ' if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes .",
    "in other words , don't dismiss this film because of its source ."
]


for i in range(len(dataset)):
    dataset[i] = dataset[i].lower()
    dataset[i] = re.sub(r'\W', ' ', dataset[i])
    dataset[i] = re.sub(r'\s+', ' ', dataset[i])


stopwords = nltk.corpus.stopwords.words('english')
for i in range(len(dataset)):
    words = nltk.word_tokenize(dataset[i])
    words = [word for word in words if word not in stopwords and not word.isdigit()]
    dataset[i] = ' '.join(words)

tfidf_vectorizer = TfidfVectorizer(stop_words='english')

tfidf_vect = tfidf_vectorizer.fit_transform(dataset)

print("TF-IDF Matrix Shape:", tfidf_vect.shape)


import pandas as pd
tfidf_tdm = pd.DataFrame(tfidf_vect.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


print("Feature Names (First 10 Words):", tfidf_vectorizer.get_feature_names_out()[:10])


print("TF-IDF Term-Document Matrix:")
print(tfidf_tdm)

TF-IDF Matrix Shape: (5, 61)
Feature Names (First 10 Words): ['80s' 'adapted' 'alan' 'arthouse' 'batman' 'book' 'books' 'brought'
 'called' 'campbell']
TF-IDF Term-Document Matrix:
        80s  adapted      alan  arthouse   batman      book    books  \
0  0.000000  0.20716  0.000000   0.20716  0.20716  0.167135  0.20716   
1  0.264426  0.00000  0.264426   0.00000  0.00000  0.000000  0.00000   
2  0.000000  0.00000  0.000000   0.00000  0.00000  0.000000  0.00000   
3  0.000000  0.00000  0.000000   0.00000  0.00000  0.274304  0.00000   
4  0.000000  0.00000  0.000000   0.00000  0.00000  0.000000  0.00000   

    brought    called  campbell  ...  starters  starting   subject  success  \
0  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.20716   
1  0.264426  0.264426  0.213337  ...  0.264426  0.000000  0.000000  0.00000   
2  0.000000  0.000000  0.208642  ...  0.000000  0.258607  0.258607  0.00000   
3  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fatimashamilova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fatimashamilova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:

from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pandas as pd
import nltk
import re


client = MongoClient("mongodb://localhost:27017")
db = client["nlp_project_db"]
collection = db["movie_reviews"]


data = pd.DataFrame(collection.find({}, {"text": 1, "tag": 1}))


nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in nltk.corpus.stopwords.words('english') and not word.isdigit()]
    return ' '.join(words)

data["processed_text"] = data["text"].apply(preprocess_text)


vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(data["processed_text"])
y = data["tag"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = MultinomialNB()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fatimashamilova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fatimashamilova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

         neg       0.71      0.66      0.69      6371
         pos       0.70      0.74      0.72      6573

    accuracy                           0.70     12944
   macro avg       0.71      0.70      0.70     12944
weighted avg       0.71      0.70      0.70     12944



In [25]:

nb_classifier = MultinomialNB()


nb_classifier.fit(X_train, y_train)


y_pred = nb_classifier.predict(X_test)

print("Predictions:", y_pred[:5])


Predictions: ['pos' 'neg' 'pos' 'neg' 'neg']


In [26]:

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         neg       0.71      0.66      0.69      6371
         pos       0.70      0.74      0.72      6573

    accuracy                           0.70     12944
   macro avg       0.71      0.70      0.70     12944
weighted avg       0.71      0.70      0.70     12944



In [30]:
pip install pymongo dnspython

Note: you may need to restart the kernel to use updated packages.


In [32]:
import nltk
from nltk.corpus import stopwords
import re


nltk.download('punkt')
nltk.download('stopwords')


stop_words = stopwords.words('english')

def preprocess_text(text):
    text = text.lower() 
    text = re.sub(r'\W', ' ', text)  
    text = re.sub(r'\s+', ' ', text) 
    words = nltk.word_tokenize(text)  
    words = [word for word in words if word not in stop_words] 
    return ' '.join(words)

data['processed_text'] = data['text'].apply(preprocess_text)

print(data[['text', 'processed_text']].head())

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fatimashamilova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fatimashamilova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  \
0  films adapted from comic books have had plenty...   
1  for starters , it was created by alan moore ( ...   
2  to say moore and campbell thoroughly researche...   
3  the book ( or " graphic novel , " if you will ...   
4  in other words , don't dismiss this film becau...   

                                      processed_text  
0  films adapted comic books plenty success wheth...  
1  starters created alan moore eddie campbell bro...  
2  say moore campbell thoroughly researched subje...  
3  book graphic novel 500 pages long includes nea...  
4                          words dismiss film source  


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_text'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print(tfidf_df.head())

    00  000  0009f  007  00s   03   04   05  05425   10  ...  zukovsky  zulu  \
0  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  ...       0.0   0.0   
1  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  ...       0.0   0.0   
2  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  ...       0.0   0.0   
3  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  ...       0.0   0.0   
4  0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  ...       0.0   0.0   

   zundel  zurg  zus  zweibel  zwick  zwigoff  zycie  zzzzzzz  
0     0.0   0.0  0.0      0.0    0.0      0.0    0.0      0.0  
1     0.0   0.0  0.0      0.0    0.0      0.0    0.0      0.0  
2     0.0   0.0  0.0      0.0    0.0      0.0    0.0      0.0  
3     0.0   0.0  0.0      0.0    0.0      0.0    0.0      0.0  
4     0.0   0.0  0.0      0.0    0.0      0.0    0.0      0.0  

[5 rows x 39511 columns]


In [1]:
#!/usr/bin/env python3
#-*- coding: utf-8 -*-

# import the MongoClient class from the library
from pymongo import MongoClient

# create an client instance of the MongoDB class
mongo_client = MongoClient()

# create an instance of 'some_database'
db = mongo_client.MovieReviewDB


# access a MongoDB collection name with spaces
col = db["movie_review"]
print ("Some Collection:", col)

Some Collection: Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'MovieReviewDB'), 'movie_review')
